# Linear estimators

In this section, we take advantage of some of `scikit-learn` powerful features such as the `pipeline`. 

In [ ]:
#hide 
%load_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import numpy as np
from tqdm.auto import tqdm

from ml4pmt.plot import line, heatmap


from matplotlib import pyplot as plt 

## Ridge / Lasso / Elastic net

Ridge regression: the betas $\langle \beta_1, ..., \beta_p \rangle$ are chosen to minimize 
$$
\frac{1}{2} \sum_{i=1}^N (y_i - \beta_0 - \sum_{j=1}^p x_{ij} \beta_j)^2 + \lambda \sum_{j=1}^p \beta_j^2
$$
The Ridge regression provides more stable and accurate estimates than standard residual sum of squares minimization

Lasso regression: the betas $\langle \beta_1,...,\beta_p \rangle$ are chosen to minimize 
$$
\frac{1}{2} \sum_{i=1}^N (y_i - \beta_0 - \sum_{j=1}^p x_{ij} \beta_j)^2 + \lambda \sum_{j=1}^p |\beta_j|
$$
The Lasso tends to promote sparse and stable models that can be more easily interpretable. 


Elastic net: the betas $\langle \beta_1,... ,\beta_p \rangle$ are chosen to minimize 
$$
	\frac{1}{2} \sum_{i=1}^N (y_i - \beta_0 - \sum_{j=1}^p x_{ij} \beta_j)^2 + \lambda \sum_{j=1}^p [(1-\alpha) \beta_j^2 + \alpha |\beta_j|]
$$

>  ``The lasso penalty is not very selective in the choice among a set of strong but correlated predictors, and the ridge penalty is inclined to shrink the coefficients of correlated variables towards each other. The compromise in the elastic net could cause the highly correlated features to be averaged while encouraging a parsiminuous model."

To give an example, we use a diabetes dataset provided by `sklearn`.

In [ ]:
from sklearn import datasets

In [ ]:
X, y = datasets.load_diabetes(return_X_y=True, as_frame=True)

In [ ]:
from sklearn.linear_model import lasso_path, enet_path
from sklearn import datasets
X, y = datasets.load_diabetes(return_X_y=True,  as_frame=True)
X /= X.std(axis=0)  
eps = 5e-3  

In [ ]:
alphas_lasso, coefs_lasso, _ = lasso_path(X, y, eps=eps)
l1_ratio = 0.5
alphas_enet, coefs_enet, _ = enet_path(X, y, eps=eps, l1_ratio=l1_ratio)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 6))
fig.suptitle('Coefficients as a function of the shrinkage factor (in log)')
line(pd.DataFrame(coefs_lasso.T, -1*np.log(alphas_lasso), columns=X.columns), title='Lasso', ax=ax[0])
line(pd.DataFrame(coefs_enet.T, -1*np.log(alphas_enet), columns=X.columns), 
     title=f'Elastic net (l1_ratio={l1_ratio})', ax=ax[1])

See more discussion in 

> Hastie, Trevor, et al. *The elements of statistical learning: data mining, inference, and prediction.* Vol. 2. New York: springer, 2009.

## Revisiting the industry backtest with linear models

In the `Industry momentum` section, we introduced a feature `X` as 12-month trailing past returns and target `y` as the one-month forward return. For $N$ industries (e.g. $N=12$), `X` and `y` are vectors of size $N$. 

In this section, we are interested in fitting linear models (potentially with regularisation à la Ridge, Lasso or Elastic Net) such that 

$$ y = B X, $$ 

where $B$ is a matrix of size $[N \times N]$. (In this equation, we ignore intercept terms which are generally small.) This equation falls into two lines of research: 

- in classic Econometrics, this is called a Vector Autoregressive (VAR) model (.e.g see the [wikipedia article](https://en.wikipedia.org/wiki/Vector_autoregression)). 

- in Machine-learning, this is often called a `multioutput` model. 

The interpretation of this model in the context of industry timing is that it captures cross-industry effect: 

- for instance, a increasing trend on oil prices (which is positive for the Energy stocks) might be a negative predictor for sectors that use oil as input (e.g. Manufacturing)

Such VAR/multioutput models will pick up the positive lead-lag correlations across industries and therefore potentially enrich the simple `Industry momentum` strategy that we introduced in the first section.

### Scikit-learn Pipeline and Multi-output

From the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html), the definition of a `sklearn` pipeline is: 

> Sequentially apply a list of transforms and a final estimator. Intermediate steps of the pipeline must be ‘transforms’, that is, they must implement fit and transform methods. The final estimator only needs to implement fit. 

Estimator like linear regressions or `Ridge` regressions have the property to estimate with `multiouput` as the regression above, but are notheless estimators, not transormations. However, 


- in the pipelines that we are building, the last step is the `MeanVariance` class that produces the holdings;  

- in the `sklearn` pipelines, all the steps except for the last one must be transformations; 

- despite formally not having a `transform` function, the `multioutput` linear estimators (such as `Ridge` and `Lasso`) are transformations of a vector `X` of size $N$ into a vector `y` of size $N$. 

In the following module, we extend the estimators that we will be using to have such `transform` property. 

In [ ]:
%%writefile ../ml4pmt/estimators.py
from sklearn.neural_network import MLPRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV



class LinearRegression(LinearRegression):
    def transform(self, X):
        return self.predict(X)


class Ridge(Ridge):
    def transform(self, X):
        return self.predict(X)


class RidgeCV(RidgeCV):
    def transform(self, X):
        return self.predict(X)


class MultiOutputRegressor(MultiOutputRegressor):
    def transform(self, X):
        return self.predict(X)


class MLPRegressor(MLPRegressor):
    def transform(self, X):
        return self.predict(X)

### Linear Regression

We first load the data and the main custom functions to run a backtest. 

In [ ]:
from ml4pmt.backtesting import MeanVariance, Backtester
from ml4pmt.dataset import load_kf_returns
returns_data = load_kf_returns(cache_dir='data')

ret = returns_data['Monthly']['Average_Value_Weighted_Returns'][:'1999']

transform_X = lambda x: x.rolling(12).mean().fillna(0).values
transform_y = lambda x: x.shift(-1).values
features = transform_X(ret)
target = transform_y(ret)

As a reference, we re-compute the pnl of a simple `Industry momentum` strategy.

In [ ]:
m = Backtester(MeanVariance(), ret).train(features, target)
pnls = {'momentum':  m.pnl_ }

We now load the functions that are specific to building `sklearn` pipelines. 

In [ ]:
from ml4pmt.estimators import Ridge, LinearRegression

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

In the following simple pipeline, there are two steps: 

- the first step is a simple `multioutput` linear regression that produces fitted predictions for each industry

- the second step takes these predictors and scales them as holdings with a mean-variance optimisation (with the `MeanVariance` class introduced earlier. 

In [ ]:
estimator = make_pipeline(LinearRegression(), MeanVariance())

In [ ]:
m = Backtester(estimator, ret).train(features, target)
pnls['linear_regression'] = m.pnl_ 
line(pnls['linear_regression'], cumsum=True, title='Linear Regression')

The linear regression fits an intercept and some coefficients. 

In [ ]:
ols_ = m.estimators_[0].named_steps['linearregression']
coef_ = ols_.coef_
intercept_ = ols_.intercept_
vec = ret.mean().values
np.allclose(ols_.predict(vec[None, :]), coef_.dot(vec)  + intercept_)

In [ ]:
coefs_ = [m.named_steps['linearregression'].coef_ for m in m.estimators_]
coefs_mean = pd.DataFrame(sum(coefs_)/len(coefs_), ret.columns, ret.columns).T

In [ ]:
heatmap(coefs_mean.loc[coefs_mean.mean(1).sort_values().index, coefs_mean.mean(1).sort_values().index],
       title='Average linear regression coefficients (x-axis: predictors, y-axis=targets)')

In [ ]:
pnls_ = {}
for hl in tqdm([6, 12, 24]): 
    features_ = ret.ewm(halflife=hl).mean().fillna(0).values
    pnls_[hl] = Backtester(estimator, ret).train(features_, target).pnl_
line(pnls_, cumsum=True, title='Robustness on feature half-lives')

In [ ]:
pnls_ = {}
for hl in [6, 12, 24]: 
    features_ = ret.rolling(window=hl).mean().fillna(0).values
    pnls_[hl] = Backtester(estimator, ret).train(features_, target).pnl_
line(pnls_, cumsum=True, title='Robustness on features with rolling windows')

### Ridge

Given that the `Ridge` regression applies a constraint across features, each feature needs to be properly rescaled, which is done here with `StandardScalar`.

In [ ]:
estimator = make_pipeline(StandardScaler(with_mean=False), 
                          Ridge(), 
                          MeanVariance())

pnls['ridge']= Backtester(estimator, ret).train(features, target).pnl_
line(pnls['ridge'], cumsum=True, title='Ridge')

In [ ]:
pnls_ = {}
for alpha in [0.1, 1, 10, 100]: 
    estimator_ = make_pipeline(StandardScaler(with_mean=False), Ridge(alpha=alpha), MeanVariance())
    pnls_[alpha] = Backtester(estimator_, ret).train(features_, target).pnl_
line(pnls_, cumsum=True, title='Ridge: Robustness on alpha')

### Ridge with feature expansion

We can expand the set of features by using polynomial transfomrs with `PolynomialFeatures`.

In [ ]:
PolynomialFeatures(degree=2).fit_transform(ret.iloc[:10]).shape

Number of new features: intercept, initial features (=12), squared features (12), all cross features of degree 1 (=6*12): 

In [ ]:
estimator = make_pipeline(StandardScaler(with_mean=False), 
                          PolynomialFeatures(degree=2), 
                          Ridge(alpha=100), 
                          MeanVariance())

In [ ]:
print(f'Number of features generated by degree=2: {1+ 12 + 12 + 6 * 11}')

In [ ]:
pnls['ridge_with_feature_expansion'] = Backtester(estimator, ret).train(features_, target).pnl_
line(pnls['ridge_with_feature_expansion'], cumsum=True, title='Ridge with feature extension')

In [ ]:
pnls_ = {}
for alpha in [0.1, 1, 10, 100, 1000]: 
    estimator_ = make_pipeline(StandardScaler(with_mean=False), 
                              PolynomialFeatures(degree=2), 
                              Ridge(alpha=alpha), 
                              MeanVariance())
    pnls_[alpha] = Backtester(estimator_, ret).train(features_, target).pnl_
line(pnls_, cumsum=True, title='Ridge with feature expansion: Robustness on alpha')

Putting all the types of linear predictors together, we can compare the cumulative pnls in the graph below.

In [ ]:
line(pd.concat(pnls, axis=1).assign(ALL = lambda x: x.mean(axis=1)), cumsum=True)